# лабораторна робота по ИИ

## 1) один нейрон

In [2]:
import pandas as pd
import numpy as np
from itertools import product

In [2]:
def neuron(x=[7,4,5], y=0.3, delta_error=0.1):     
    
    if len(x)!=3:
        print('error len of x')
        return 
    if type(y) is not float and type(y) is not int:
        print('error y type is uncorrect')
        return 
    if type(delta_error) is not float and type(delta_error) is not int:
        print('error delta_error type is uncorrect')
        return 
    
    
    # initialization var
    x = np.array([1] + x)
    w = np.random.normal(loc=0.1, scale=0.05, size=(len(x), 1))
    df = pd.DataFrame(columns=['w0', 'w1', 'w2', 'w3', 'y_predict', 'delta'])
    
    def predict(x_list):
        x_list = np.array([1] + x_list)
        s = np.dot(x_list, w)[0]
        y = activization_function(s)
        return y
    
    while True:
        
        s_m = np.dot(x, w)[0]
        y_m = activization_function(s_m)
        
        delta = abs((y_m - y) / y)
        df = df.append({'w0':w[0][0], 'w1':w[1][0], 'w2':w[2][0], 'w3':w[3][0], 'y_predict':y_m, 'delta': delta}, 
                       ignore_index=True)
        
        if delta <= delta_error: 
            return df, predict
        
        eta = y_m*(1-y_m)
        delta_w = eta * (y - y_m) * x
        w = w + delta_w.reshape(len(x), 1)
        

In [12]:
def activization_function(sum_x):
    return 1/(1 + np.exp(-sum_x))

In [4]:
df, prd = neuron()
df[-10:]

,w0,w1,w2,w3,y_predict,delta
0,0.090455,0.131097,0.124785,0.086221,0.874174,1.913915
1,0.027300,-0.310991,-0.127837,-0.229556,0.021694,0.927685
2,0.033207,-0.269644,-0.104210,-0.200023,0.036571,0.878098
3,0.042488,-0.204674,-0.067084,-0.153615,0.081164,0.729455
4,0.058808,-0.090434,-0.001805,-0.072016,0.280592,0.064693


In [5]:
prd([7,4,5])

0.2805921931772059

## 2) Многослойный перцептрон, алгоритм обратного распространения ошибки 1\*1\*1

In [11]:
def Multilayer_perceptron(x=7, y=0.3, delta_error=0.01):
    if type(x) is not float and type(x) is not int:
        print('error x type is uncorrect')
        return 
    if type(y) is not float and type(y) is not int:
        print('error y type is uncorrect')
        return 
    if type(delta_error) is not float and type(delta_error) is not int:
        print('error delta_error type is uncorrect')
        return 
    
    df = pd.DataFrame(columns=['delta', 'y_pr'])
    w12, w23 = np.random.normal(loc=0.1, scale=0.05, size=2)
    
    def predict(x_pr):
        x_s1 = x_pr * w12
        y = activization_function(x_s1)
        
        x_s2 = y * w23
        return activization_function(x_s2)
    
    while True:
        x_sum1 = x * w12
        y2 = activization_function(x_sum1)
        
        x_sum2 = y2 * w23
        y3 = activization_function(x_sum2)
        
        delta = abs((y3 - y) / y)
    
        df = df.append({'delta':delta, 'y_pr':y3}, ignore_index=True)
        
        if delta <= delta_error:
            return df, predict
        
        sigma_3 = y3 * (1-y3) * (y - y3)
        delta_w23 = y2 * sigma_3
        
        sigma_2 = y2 * (1-y2) * (sigma_3 * w23)
        delta_w12 = x * sigma_2
        
        w12 += delta_w12
        w23 += delta_w23
        

In [12]:
df, prd = Multilayer_perceptron()
df[-10:]

,delta,y_pr
86,0.014571,0.304371
87,0.013924,0.304177
88,0.013306,0.303992
89,0.012716,0.303815
90,0.012152,0.303646
91,0.011614,0.303484
92,0.011099,0.303330
93,0.010608,0.303182
94,0.010139,0.303042
95,0.009691,0.302907


In [13]:
prd(6.66)

0.30428897196337795

## 2) многослойный песептрон с одним скрытым слоем

In [31]:
def Multilayer_perceptron_2(x=[2,4,7,3], y=0.3, hidden_layer=40, w1=[], w2=[]):
#     print(pd.DataFrame(w1, columns=['neuron1', 'neuron2', 'neuron3'], index=['x0','x1','x2']))
#     print(pd.DataFrame(w2, index=['y0','neuron1', 'neuron2', 'neuron3'], columns=['exit']))
    x = [1] + list(x)
    y_hidden = np.array([1] + list(activization_function(np.dot(x, w1))))
    y_predict = activization_function(np.dot(y_hidden, w2))
#     print('x\n',x,'\ny_hidden\n',y_hidden, '\ny_predict\n', y_predict)

    delta_exit = y_predict * (1 - y_predict) * (y - y_predict)
    delta_hidden = y_hidden * (1 - y_hidden) * (delta_exit * w2.reshape(hidden_layer + 1))
    
#     print('delta_exit\n',delta_exit,'\delta_hidden\n',delta_hidden)

    
    for i in range(len(x)):
        w1[i,:] += x[i] * delta_hidden[1:]
    
    w2 += y_hidden.reshape(hidden_layer+1, 1) * delta_exit
#     print(pd.DataFrame(w1, columns=['neuron1', 'neuron2', 'neuron3'], index=['x0','x1','x2']))
#     print(pd.DataFrame(w2, index=['y0','neuron1', 'neuron2', 'neuron3'], columns=['exit']))
    return w1, w2
        
        

In [36]:
def online_perceptron(x_list=[[1,2],[2,1],[1,1]], y_list=[0.03, 0.03, 0.02], 
                     count_iter=1000):
    
    hidden_layer = 3
    exit_layer = 1

    w1 = np.random.normal(loc=0.1, scale=0.05, size=(len(x_list[0]) + 1, hidden_layer))
    w2 = np.random.normal(loc=0.1, scale=0.05, size=(hidden_layer + 1, exit_layer))

    def predict(x_pr):
        y_h = np.array([1] + list(activization_function(np.dot([1] + x_pr, w1))))
        return activization_function(np.dot(y_h, w2))
    
    for i in range(count_iter):
        for x, y in zip(x_list, y_list):
            x = list(x)
            w1, w2 = Multilayer_perceptron_2(x=x, y=y, w1=w1, w2=w2, hidden_layer=hidden_layer)
            
    print(predict(x), x , y)
    return predict

In [1]:
x, y = np.arange(0, 5, 0.2), np.arange(0, 5, 0.2)
fun = lambda x,y: x+y
f = [fun(xi,yi) for xi, yi in product(x,y)]


prd = online_perceptron(x_list=list(product(x,y)), y_list=f, count_iter=1000)

NameError: name 'np' is not defined

# debug 





In [24]:
activization_function(0.12+0.133*0.52+0.205*0.525+0.105*0.527)

0.5871315788168427

In [25]:
0.587*(1-0.587)*(0-0.587)

-0.142306997

In [26]:
-0.1423*1+0.12, -0.1423*0.52+0.133, -0.1423*0.525+0.205, -0.1423*0.527+0.105

(-0.022300000000000014, 0.059004, 0.13029249999999998, 0.03000789999999999)

In [ ]:
prd([1,2]), prd([1,1]), prd([1,3])

In [38]:
prd = online_perceptron()
prd([1,1]), prd([2,1]), prd([1,2])

[0.02927067] [1, 1] 0.02


(array([0.02927067]), array([0.02482367]), array([0.02509714]))